# Obstruction Detection - Proposed Method with GPT-4o

only use the mask with highest confidence

detection accuracy = 94.11%

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import sys
import base64
import requests
from PIL import Image
import io
import json
import uvicorn
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import FileResponse, StreamingResponse, JSONResponse
from PIL import Image, ImageOps, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import numpy as np
import io
import subprocess
import torch
import json
import os
import warnings
import numpy as np
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO
from lang_sam import LangSAM
import torch
import cv2
import base64
import openai
from openai import OpenAI
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import queue
import subprocess

In [2]:
root_dir = '...'

mask_dir = root_dir + 'augmented content masks/' # this is the mask of ar content
ar_dir = root_dir + 'augmented images/'
img_dir = root_dir + 'raw img all/'

# data_csv = '/home/yanming/Workspace/Detrimental Detection/Dataset Folder v1 - 119 imgs/adjusted_dataset.csv'
ar_csv = '.../dataset-gt-no-laptop.csv'

# data = pd.read_csv(data_csv)
ar_data = pd.read_csv(ar_csv)

print(len(os.listdir(mask_dir)))
print(len(os.listdir(ar_dir)))
print(len(os.listdir(img_dir)))

print(len(ar_data))


322
322
322
306


### Main Testing Part

In [ ]:
model = LangSAM(sam_type="vit_b")

result_csv = '...'

## GPT part

In [4]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# account 1: google login
os.environ["OPENAI_API_KEY"] = "..."
api_key = '...'

question = "Identify the key object that need people's attention or safety inspection in the image. \
            Give only one object that you think is important to be noticed, \
            output its name only and do not provide any other information. \
            The objects can be caution information signs, electrical devices, safety equipments, etc. \
            If you think a color is important, you can also mention the color, such as red box, but \
            try to be precise and describe the object with no more than 4 words. "   
            

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

In [ ]:
correct = 0
total = 0

tp = 0
tn = 0
fp = 0
fn = 0
no_prediction = 0

rand_thres_flag = True
largest_flag = True

for i in range(len(ar_data)):
    
    # random threshold range from 0.15 to 0.25
    if rand_thres_flag == False:
        rand_thres = 0.2
    else:
        rand_thres = np.random.uniform(0.175, 0.225)
    
    correct = 0
    
    image_name = ar_data['File Name'][i]
    
    # print(image_name)
    
    ar_mask_name = ar_data['File Name'][i] # Help_station_4.png
    ar_mask_name = 'mask_AR_' + ar_mask_name # AR_mask_Help_station_4.png

    img = Image.open(os.path.join(img_dir, image_name))
    
    img_enc = encode_image(os.path.join(img_dir, image_name))
    
    payload = {
        # ! This is the model for GPT-4 Vision
        # "model": "gpt-4-vision-preview",
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": question
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{img_enc}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 300
    }
    
    bad_response = True
    bad_response_list = ['sorry', 'however', 'there is no', 'there are no', \
    'too', 'blurred', 'blurry', 'can\'t', 'is not', 'isn\'t', 'it\'s', 'difficult']
    
    while bad_response:

        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        
        # ! if there's no 'error' in the keys:
        if 'error' in response.json().keys():
            print('Error!')
            print(response.json())
            continue
        text_response = response.json()['choices'][0]['message']['content']
        
        # remove " and ' in the response
        text_response = text_response.replace('\"', '')
        text_response = text_response.replace("\'", '')
        
        if any(bad in text_response for bad in bad_response_list) == False:
            bad_response = False
            break
    
    
    # prompt = prompt_data[prompt_data['File Name'] == image_name]['Prompt'].values[0]
    prompt = text_response
    
    
    
    
    
    # this mask is for AR content
    ar_mask = Image.open(os.path.join(mask_dir, ar_mask_name))
    
    file_name = ar_data['File Name'][i]
    
    
    true_object = ar_data['Object Name'][i]
    
    ar_name = ar_data['File Name'][i]
    
    truth = ar_data[ar_data['File Name'] == ar_name]['obstruction'].values[0]
    truth = True if truth == 'yes' else False
    
    sam_masks, sam_boxes, sam_phrases, sam_logits = model.predict(img, prompt)
    
    
    # print(sam_logits)
    # print(sam_phrases) 
    
    
    if len(sam_masks) == 0:
        print(f"No objects of the '{prompt}' prompt detected in the image.")
        no_prediction += 1
        correct = 0
        # write the result to the csv
        with open(result_csv, 'a') as f:
            f.write(f"{file_name},{true_object},{prompt},{'N/A'},{'N/A'},{correct}\n")
            
        continue
    
    else:
        if largest_flag:
            # only leave the mask with the highest confidence
            max_logit = max(sam_logits)
            sam_masks = [mask for mask, logit in zip(sam_masks, sam_logits) if logit == max_logit]
            boxes = [box for box, logit in zip(sam_boxes, sam_logits) if logit == max_logit]
            phrases = [phrase for phrase, logit in zip(sam_phrases, sam_logits) if logit == max_logit]
            sam_logits = [logit for logit in sam_logits if logit == max_logit]
            
        mean_confidence = np.mean(sam_logits)
        sam_masks_np = [m.squeeze().cpu().numpy() for m in sam_masks]
    
        # generate a overall mask
        overall_mask = np.zeros_like(sam_masks_np[0])
        for m in sam_masks_np:
            overall_mask += m
            
        # this mask is for real objects
        overall_mask[overall_mask > 1] = 1
        
        # calculate the intersection
        intersection = np.logical_and(overall_mask, ar_mask)
        real_area = np.sum(overall_mask)
        
        if intersection.sum() / real_area > rand_thres:
            pred = True
        else:
            pred = False
            
        if pred == truth:
            correct = 1
            total += 1
        else:
            correct = 0
            
        if pred == True and truth == True:
            tp += 1
        elif pred == False and truth == False:
            tn += 1
        elif pred == True and truth == False:
            fp += 1
        elif pred == False and truth == True:
            fn += 1
        
        # write the result to the csv
        with open(result_csv, 'a') as f:
            f.write(f"{file_name},{true_object},{prompt},{mean_confidence},{truth},{pred},{correct}\n")
            
        # print(f"Prediction: {pred}, Truth: {truth}")
        

        
        
    
    
    # if i % 10 == 0:
    #     print(f"Finished {i} images.")
    # if i == 0:
    #     break
    
# print(f"Accuracy: {total / len(ar_data)}")
# print(f"TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}, No Prediction: {no_prediction}")


In [6]:
# read the csv file
df = pd.read_csv(result_csv)

print(len(df))

tp = 0
tn = 0
fp = 0
fn = 0
no_prediction = 0

print(df['Prediction'][20])
print(df['Score'][0])

print(type(df['Prediction'][20]))
print(type(df['Score'][0]))


for i in range(len(df)):
    # print(df['Prediction'][i])
    # print(df['Score'][i])
    
    if str(df['Prediction'][i]) == 'nan':
        no_prediction += 1
    elif df['Prediction'][i] == 'True' and df['Score'][i] == 1:
        tp += 1
    elif df['Prediction'][i] == 'False' and df['Score'][i] == 1:
        tn += 1
    elif df['Prediction'][i] == 'True' and df['Score'][i] == 0:
        fp += 1
    elif df['Prediction'][i] == 'False' and df['Score'][i] == 0:
        fn += 1
        
print(f"TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}, No Prediction: {no_prediction}")
print(f"Accuracy: {(tp + tn) / len(df)}")

306
False
1.0
<class 'str'>
<class 'numpy.float64'>
TP: 134, TN: 154, FP: 5, FN: 11, No Prediction: 0
Accuracy: 0.9411764705882353
